In [ ]:
!pip install "tf-models-official==2.13.*"

In [ ]:
!pip install -U "tensorflow-text==2.13.*"

In [46]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [47]:
import pandas as pd
from datetime import timedelta
from datetime import datetime
import string
import numpy as np
import os

In [48]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns

In [49]:


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,Dropout
from tensorflow.keras.metrics import MSE
from sklearn.metrics import confusion_matrix, classification_report

#import keras_nlp


In [50]:

X_train= pd.read_csv('X_train.csv')
#y_train= pd.read_csv('y_train.csv')

X_val= pd.read_csv('X_val.csv')
#y_val= pd.read_csv('y_val.csv')

X_test= pd.read_csv('X_test.csv')
#y_test= pd.read_csv('y_test.csv')

In [51]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22832 entries, 0 to 22831
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         22832 non-null  object 
 1   author                     22832 non-null  object 
 2   created                    22832 non-null  object 
 3   retrieved                  22832 non-null  object 
 4   edited                     22832 non-null  object 
 5   pinned                     22832 non-null  int64  
 6   archived                   22832 non-null  int64  
 7   locked                     22832 non-null  int64  
 8   removed                    22832 non-null  int64  
 9   deleted                    22832 non-null  int64  
 10  is_self                    22832 non-null  int64  
 11  is_video                   22832 non-null  int64  
 12  is_original_content        22832 non-null  int64  
 13  title                      22832 non-null  obj

In [52]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6338 entries, 0 to 6337
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6338 non-null   object 
 1   author                     6338 non-null   object 
 2   created                    6338 non-null   object 
 3   retrieved                  6338 non-null   object 
 4   edited                     6338 non-null   object 
 5   pinned                     6338 non-null   int64  
 6   archived                   6338 non-null   int64  
 7   locked                     6338 non-null   int64  
 8   removed                    6338 non-null   int64  
 9   deleted                    6338 non-null   int64  
 10  is_self                    6338 non-null   int64  
 11  is_video                   6338 non-null   int64  
 12  is_original_content        6338 non-null   int64  
 13  title                      6338 non-null   objec

In [53]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6337 entries, 0 to 6336
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6337 non-null   object 
 1   author                     6337 non-null   object 
 2   created                    6337 non-null   object 
 3   retrieved                  6337 non-null   object 
 4   edited                     6337 non-null   object 
 5   pinned                     6337 non-null   int64  
 6   archived                   6337 non-null   int64  
 7   locked                     6337 non-null   int64  
 8   removed                    6337 non-null   int64  
 9   deleted                    6337 non-null   int64  
 10  is_self                    6337 non-null   int64  
 11  is_video                   6337 non-null   int64  
 12  is_original_content        6337 non-null   int64  
 13  title                      6337 non-null   objec

In [54]:
X_train['all_words']=X_train['title']+' '+ X_train['selftext']
X_test['all_words']=X_test['title']+' '+ X_test['selftext']
X_val['all_words']=X_val['title']+' '+ X_val['selftext']

In [58]:
ind=X_train[X_train['engagement_score_std'].isna()].index
X_train=X_train.drop(ind)

In [59]:
X_val[X_val['engagement_score_std'].isna()].index

Index([], dtype='int64')

In [60]:
X_test[X_test['engagement_score_std'].isna()].index

Index([], dtype='int64')

In [61]:
y_train=X_train['engagement_score_std'].values
y_val=X_val['engagement_score_std'].values
y_test=X_test['engagement_score_std'].values

X_train=X_train['all_words'].apply(lambda x: str(x)).values
X_val=X_val['all_words'].apply(lambda x: str(x)).values
X_test=X_test['all_words'].apply(lambda x: str(x)).values


In [62]:
bert_model_name = 'small_bert/bert_en_uncased_L-2_H-256_A-4'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
def build_reg_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='reg')(net)

  return tf.keras.Model(text_input, net)

reg_model = build_reg_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
reg_model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

history = reg_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
125/714 [====>.........................] - ETA: 14:24 - loss: 2.4920 - mse: 2.4920

In [ ]:
def build_reg_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='reg')(net)

  return tf.keras.Model(text_input, net)

reg_model = build_reg_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
reg_model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

history = reg_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
925/925 [==============================] - 1813s 2s/step - loss: 1.0939 - mse: 1.0939 - val_loss: 0.6412 - val_mse: 0.6412
Epoch 2/5
339/925 [=========>....................] - ETA: 17:01 - loss: 0.6409 - mse: 0.6409

In [ ]:
learning_rates = 0.00001 # np.linspace(0.00001, 0.1,5)
batch_size=100
epochs=5

text_input =tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']


# Neural network layers
net=layers.Dropout(0.1)(net)
l = tf.keras.layers.Dense(128, activation='relu', name='h1')(net)
l = tf.keras.layers.Dense(64, activation='relu',name='h2')(l)
l = tf.keras.layers.Dense(32, activation='relu', name='h3')(l)
l = tf.keras.layers.Dense(16, activation='relu', name='h4')(l)

l =tf.keras.layers.Dense(1,name='output')(l)

model=tf.keras.Model(inputs=[text_input],outputs=[l])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rates)

model.compile(optimizer= optimizer, loss='mse', metrics=['mse'])

hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))


Epoch 1/5
515/515 [==============================] - 1123s 2s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/5
515/515 [==============================] - 1067s 2s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/5
515/515 [==============================] - 1070s 2s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/5
515/515 [==============================] - 1067s 2s/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/5
 14/515 [..............................] - ETA: 15:53 - loss: nan - mse: nan